In [16]:
# imports
import requests
import pandas as pd
import os

#EBIKES DATAFRAME

In [24]:
url = "http://api.citybik.es/"
href = "/v2/networks/"
#network_id = 'velib'
response = requests.get(url + href)
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response into a Python dictionary
    data = response.json()
else:
    print("Request failed with status code:", response.status_code)
networks = data['networks']
city = 'Vancouver'
filtered_network = [network for network in networks if network['location']['city'] == city]
network_id = filtered_network[0]['href']
response1 = requests.get(url + network_id)
if response1.status_code == 200:
    # Parse the JSON response into a Python dictionary
    data1 = response1.json()
else:
    print("Request failed with status code:", response.status_code)
vancouver_network_stations = pd.json_normalize(data=data1, record_path=['network', 'stations'])
vancouver_network_stations = vancouver_network_stations[['free_bikes', 'id', 'longitude', 'latitude']]
vancouver_network_stations

,free_bikes,id,longitude,latitude
0,7,7a19c49f486d7c0c02b3685d7b240448,-123.114397,49.262487
1,6,32603a87cfca71d0f7dfa3513bad69d5,-123.121817,49.274566
2,2,6d42fa40360f9a6b2bf641c7b8bb2862,-123.110154,49.279764
3,13,66f873d641d448bd1572ab086665a458,-123.113504,49.260599
4,2,485d4d24c803cfde829ab89699fed833,-123.117772,49.264215
...,...,...,...,...
247,9,10acadd77bc21f26a7f09d8c74d4af2d,-123.114237,49.277773
248,1,58345bb719f4e570ed4edc9dfa6a68a2,-123.254307,49.263073
249,5,261efc67071963969b3f848c66454d97,-123.250902,49.259999
250,6,ee620d77724c8993b0d366e7ec655b64,-123.236210,49.254780


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [18]:
#Testing to see if get request works
radius_meters = 1000
url = url = "https://api.foursquare.com/v3/places/search"
four_square_api_key = os.environ['FOURSQUARE_API_KEY']
headers = {'Accept': 'application/json'}
headers['Authorization'] = four_square_api_key
params = {
    'll': "49.262487,-123.114397",
    'fields': 'fsq_id,name,location,categories,distance,description,price,rating',
    'radius': radius_meters
}

result = requests.get(url, headers=headers, params=params)

# Check if the request was successful (status code 200)
if result.status_code == 200:
# Parse the JSON response into a Python dictionary
    fsq_json = result.json()
    print(fsq_json)
else:
    print("Request failed with status code:", result.status_code)


{'results': [{'fsq_id': '4e223f06d4c0d32590f80ff4', 'categories': [{'id': 13306, 'name': 'Taco Restaurant', 'short_name': 'Tacos', 'plural_name': 'Taco Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_', 'suffix': '.png'}}], 'distance': 164, 'location': {'address': '2450 Yukon St', 'address_extended': 'at W Broadway', 'country': 'CA', 'cross_street': 'at W Broadway', 'formatted_address': '2450 Yukon St (at W Broadway), Vancouver BC V5Y 0A4', 'locality': 'Vancouver', 'postcode': 'V5Y 0A4', 'region': 'BC'}, 'name': 'La Taqueria Pinche Taco Shop', 'price': 2, 'rating': 8.5}, {'fsq_id': '4aa73d25f964a520704c20e3', 'categories': [{'id': 17069, 'name': 'Grocery Store', 'short_name': 'Grocery Store', 'plural_name': 'Grocery Stores', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}}, {'id': 17071, 'name': 'Health Food Store', 'short_name': 'Health Food Store', 'plural_name': 'Health Food Stores', 'icon': {'pref

In [28]:
df = pd.DataFrame(fsq_json['results'])
df.head()

,fsq_id,categories,description,distance,location,name,rating,price
0,4c0b1f677e3fc928cdebf482,"[{'id': 16003, 'name': 'Beach', 'short_name': ...",Second Beach in Stanley Park is located at the...,25,"{'address': '8501 Stanley Park Dr', 'country':...",Second Beach,9.1,NaN
1,4bf046d53a002d7fb4eb85a4,"[{'id': 18075, 'name': 'Swimming Pool', 'short...","Features at Second Beach Pool; Concession, Pub...",39,"{'address': '8501 Stanley Park Dr', 'country':...",Second Beach Pool,9.1,NaN
2,4c3e8adf83a3a593e053ee33,"[{'id': 10000, 'name': 'Arts and Entertainment...",NaN,111,"{'country': 'CA', 'formatted_address': 'Vancou...",Outdoor Movies by Fresh Air Cinemas,8.1,NaN
3,4cdc9e88fc973704e4d6d605,"[{'id': 18047, 'name': 'Tennis Court', 'short_...",NaN,514,"{'country': 'CA', 'cross_street': 'at Stanley ...",Stanley Park Tennis Courts,7.9,NaN
4,4ba69381f964a520ad5f39e3,"[{'id': 16019, 'name': 'Hiking Trail', 'short_...",The Seawall is a scenic 22 km of path that lin...,623,"{'address': 'Beach Ave', 'country': 'CA', 'cro...",Stanley Park English Bay Seawall,9.6,NaN


In [41]:
fsq_df = pd.DataFrame()
radius_meters = 1000
url = url = "https://api.foursquare.com/v3/places/search"
four_square_api_key = os.environ['FOURSQUARE_API_KEY']
headers = {'Accept': 'application/json'}
headers['Authorization'] = four_square_api_key

#itterate over all the ebikes stations above using longitude and latitude to get POI
for index, row in vancouver_network_stations.iterrows():
    params = {
    'll': f"{row['latitude']},{row['longitude']}",
    'fields': 'fsq_id,name,location,categories,distance,description,price,rating',
    'radius': radius_meters
}
    result = requests.get(url, headers=headers, params=params)

# Check if the request was successful (status code 200)
    if result.status_code == 200:
    # Parse the JSON response into a Python dictionary
        fsq_json = result.json()
        df = pd.DataFrame(fsq_json['results'])
        fsq_df = pd.concat([fsq_df, df], ignore_index=True)
#else:
    print("Request failed with status code:", result.status_code)

Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
Request failed with status code: 200
R

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [43]:
fsq_df.head()

,fsq_id,categories,distance,location,name,price,rating,description
0,4e223f06d4c0d32590f80ff4,"[{'id': 13306, 'name': 'Taco Restaurant', 'sho...",164,"{'address': '2450 Yukon St', 'address_extended...",La Taqueria Pinche Taco Shop,2.0,8.5,NaN
1,4aa73d25f964a520704c20e3,"[{'id': 17069, 'name': 'Grocery Store', 'short...",182,"{'address': '510 8th Ave W', 'country': 'CA', ...",Whole Foods,3.0,8.6,"Welcome to Vancouver, BC Whole Foods Market! W..."
2,588e7f618b0ebb5d010abac4,"[{'id': 13272, 'name': 'Ramen Restaurant', 'sh...",186,"{'address': '558 Broadway W', 'country': 'CA',...",Hokkaido Ramen Santouka,NaN,7.8,NaN
3,4aa843e0f964a5207f5020e3,"[{'id': 13068, 'name': 'American Restaurant', ...",232,"{'address': '575 West Broadway', 'country': 'C...",Cactus Club Cafe Broadway + Ash,3.0,7.5,Cactus Club Cafe is the leading casual fine di...
4,4ab145f3f964a520d56820e3,"[{'id': 13002, 'name': 'Bakery', 'short_name':...",283,"{'address': '368 7th Ave W', 'country': 'CA', ...",Solly's Bagelry,2.0,7.9,NaN


In [130]:
fsq_df['categories']

def extract_values(dictionary_list):
    values = [list(d.values())[1] for d in dictionary_list if d][0]  # Extracting values from dictionaries
    return values
fsq_df['category'] = fsq_df['categories'].apply(lambda x: extract_values(x))
fsq_parsed = fsq_df[['name', 'category', 'distance', 'price', 'rating']]
fsq_parsed

,name,category,distance,price,rating
0,La Taqueria Pinche Taco Shop,Taco Restaurant,164,2.0,8.5
1,Whole Foods,Grocery Store,182,3.0,8.6
2,Hokkaido Ramen Santouka,Ramen Restaurant,186,NaN,7.8
3,Cactus Club Cafe Broadway + Ash,American Restaurant,232,3.0,7.5
4,Solly's Bagelry,Bakery,283,2.0,7.9
...,...,...,...,...,...
2509,Donald's Market,Grocery Store,501,NaN,8.7
2510,Tacofino Commissary,Mexican Restaurant,506,2.0,8.6
2511,Platform 7 Coffee,Café,507,1.0,8.4
2512,The Downlow Chicken Shack,Fried Chicken Joint,610,1.0,8.6


Put your parsed results into a DataFrame

In [132]:
fsq_parsed.head()

,name,category,distance,price,rating
0,La Taqueria Pinche Taco Shop,Taco Restaurant,164,2.0,8.5
1,Whole Foods,Grocery Store,182,3.0,8.6
2,Hokkaido Ramen Santouka,Ramen Restaurant,186,NaN,7.8
3,Cactus Club Cafe Broadway + Ash,American Restaurant,232,3.0,7.5
4,Solly's Bagelry,Bakery,283,2.0,7.9


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

PLEASE NOTE THAT THE BELOW GET REQUESTS WORKED SEVERAL DAYS AGO AND URL, PARAMERTERS, HEADERS WORK IN POSTMAN, BUT THE GET REQUEST STILL RETURNS AN ERROR. I SPENT OVER ONE HOUR WITH LARRY AI AND COULDN'T SOLVE THE PROBLEM AND OVER ONE HOUR WITH A MENTOR WHO COULDN'T SOLVE THE PROBLEM. I TRIED SEVERAL ACCOUNTS IN YELP AND I AM UNABLE TO SPEND MORE TIME TRYING TO SOLVE THIS SO I AM GOING TO USE TWO SEPERATE JSON FILES COPIED FROM POSTMAN TO MIMICK GET REQUESTS TO PROVE I CAN MANIPULATE AND DOWNLOAD INFORMATION TO A PANDAS DATAFRAME


In [60]:
import urllib.request
import urllib.parse
import json

yelp_df = pd.DataFrame()
api_key = os.environ['YELP_API_KEY']
url = "https://api.yelp.com/v3/businesses/search"

headers = {
    "Authorization": f"Bearer {api_key}"
}
    
for index, row in vancouver_network_stations.iterrows():
    params = {
        "latitude": row['latitude'],
        "longitude": row['longitude'],
        "term": "restaurants",
        "radius": 1000,
        "limit": 10
    }
    query_string = urllib.parse.urlencode(params)
    url_with_params = f"{url}?{query_string}"
    req = urllib.request.Request(url_with_params, headers=headers)

    try:
        with urllib.request.urlopen(req) as response:
            response_body = response.read()
            data = json.loads(response_body.decode("utf-8"))
            df = pd.DataFrame(data['businesses'])
            df['station_id'] = row['station_id']
            yelp_df = pd.concat([yelp_df, df], ignore_index=True) 
    except urllib.error.HTTPError as e:
        print(f"HTTP error occurred: {e.code} - {e.reason}")
    except urllib.error.URLError as e:
        print(f"URL error occurred: {e.reason}")
    


HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error occurred: 429 - Too Many Requests
HTTP error

In [62]:
yelp_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,attributes,price
0,F5wxgIiZE7LYQxgqhI483A,seaport-city-seafood-restaurant-vancouver,Seaport City Seafood Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/zRW3aV...,False,https://www.yelp.com/biz/seaport-city-seafood-...,52,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...",4.3,"{'latitude': 49.263921119674905, 'longitude': ...",[],"{'address1': '2425 Cambie Street', 'address2':...",+16042227288,+1 604-222-7288,168.615262,"{'business_temp_closed': None, 'menu_url': Non...",NaN
1,XAH2HpuUUtu7CUO26pbs4w,saku-vancouver-8,Saku,https://s3-media2.fl.yelpcdn.com/bphoto/RJV1GX...,False,https://www.yelp.com/biz/saku-vancouver-8?adju...,250,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.3,"{'latitude': 49.2631006, 'longitude': -123.116...",[],"{'address1': '548 W Broadway', 'address2': '',...",+16044234600,+1 604-423-4600,178.845344,"{'business_temp_closed': None, 'menu_url': 'ht...",$$
2,NensKn1MSVU_rm-1Y6WlFA,marulilu-cafe-vancouver,Marulilu Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/jUmSJk...,False,https://www.yelp.com/biz/marulilu-cafe-vancouv...,290,"[{'alias': 'cafes', 'title': 'Cafes'}]",4.0,"{'latitude': 49.26338, 'longitude': -123.11415}",[],"{'address1': '451 W Broadway', 'address2': '',...",+16045684211,+1 604-568-4211,102.329817,"{'business_temp_closed': None, 'menu_url': 'ht...",$$
3,nfdTh56TAqekRLz-TUFOBg,xin-chào-vancouver-2,Xin Chào,https://s3-media3.fl.yelpcdn.com/bphoto/Lp9TJm...,False,https://www.yelp.com/biz/xin-ch%C3%A0o-vancouv...,3,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",4.0,"{'latitude': 49.2626123961945, 'longitude': -1...",[],"{'address1': '2559 Cambie Street', 'address2':...",+16046959384,+1 604-695-9384,56.643752,"{'business_temp_closed': None, 'menu_url': 'ht...",NaN
4,6iOAgzJ0DRZNSKA3FSrrOg,la-taqueria-pinche-taco-shop-vancouver,La Taqueria Pinche Taco Shop,https://s3-media1.fl.yelpcdn.com/bphoto/fjhIj3...,False,https://www.yelp.com/biz/la-taqueria-pinche-ta...,685,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.2,"{'latitude': 49.263559, 'longitude': -123.112736}",[],"{'address1': '2450 Yukon Street', 'address2': ...",+16045582549,+1 604-558-2549,169.517456,"{'business_temp_closed': None, 'menu_url': '',...",$$


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [119]:
yelp_df['category'] = df['categories'].apply(lambda x: [a['alias'] for a in x][0])
yelp_df['category'] = yelp_df['category']
yelp_parsed = yelp_df[['name', 'category', 'rating', 'distance', 'price']]
yelp_parsed

,name,category,rating,distance,price
0,Seaport City Seafood Restaurant,dimsum,4.3,168.615262,NaN
1,Saku,turkish,4.3,178.845344,$$
2,Marulilu Cafe,breakfast_brunch,4.0,102.329817,$$
3,Xin Chào,japanese,4.0,56.643752,NaN
4,La Taqueria Pinche Taco Shop,taiwanese,4.2,169.517456,$$
...,...,...,...,...,...
455,Yu Shang Hot Pot,NaN,4.3,499.194875,NaN
456,Marulilu Cafe,NaN,4.0,429.294440,$$
457,Pho & Roll by Wooden Fish,NaN,3.7,319.134872,NaN
458,Branas Mediterranean Grill,NaN,3.5,382.761981,$$


Put your parsed results into a DataFrame

In [83]:
yelp_parsed

,name,alias,rating,distance,price
0,Seaport City Seafood Restaurant,"[dimsum, seafood, cantonese]",4.3,168.615262,NaN
1,Saku,"[turkish, mediterranean]",4.3,178.845344,$$
2,Marulilu Cafe,[breakfast_brunch],4.0,102.329817,$$
3,Xin Chào,[japanese],4.0,56.643752,NaN
4,La Taqueria Pinche Taco Shop,"[taiwanese, soup, hotpot]",4.2,169.517456,$$
...,...,...,...,...,...
455,Yu Shang Hot Pot,NaN,4.3,499.194875,NaN
456,Marulilu Cafe,NaN,4.0,429.294440,$$
457,Pho & Roll by Wooden Fish,NaN,3.7,319.134872,NaN
458,Branas Mediterranean Grill,NaN,3.5,382.761981,$$


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp provides more complet data: in addtion to foursquare location data yelp provides: reviews
Yelp is catered to restaurants, but Foursquare has other points of interest as well. If you are looking for details about restaurants Yelp is bettter, but for other POIs Foursquare has better coverage.
Yelp get requests also gives many results while Foursquare only returns 10 results per request
Foursquare gives you $200 credit per month - I used $12 for the project
Yelp gives 300 requests per day

Get the top 10 restaurants according to their rating

In [152]:
#foursquare top 10
fsq_filtered = fsq_parsed[fsq_parsed['category'].str.contains('Restaurant')]
fsq_sorted = fsq_filtered.sort_values(by='rating', ascending=False)
top_10 = fsq_sorted.drop_duplicates(subset=['name'])
top_10.head(10)

,name,category,distance,price,rating
1082,The Mackenzie Room,Restaurant,499,NaN,9.2
194,Le Crocodile Restaurant,French Restaurant,228,4.0,9.1
90,Hawksworth Restaurant,Restaurant,125,4.0,9.1
470,Kingyo Izakaya 金魚居酒屋,Sushi Restaurant,308,3.0,9.1
1125,Phnom Penh,Cambodian Restaurant,660,2.0,9.0
1976,Green Leaf Sushi,Sushi Restaurant,373,2.0,9.0
443,Tacofino,Taco Restaurant,193,1.0,9.0
436,Minami Restaurant,Sushi Restaurant,194,3.0,9.0
2072,Cactus Club Cafe,Mexican Restaurant,98,2.0,9.0
990,Ramen Danbo - Robson,Ramen Restaurant,131,2.0,9.0


In [156]:
#Yelp top 10
yelp_sorted = yelp_parsed.sort_values(by='rating', ascending=False)
top_10 = yelp_sorted.drop_duplicates(subset=['name'])
top_10.head(10)

,name,category,rating,distance,price
429,That’s It Cafe,NaN,5.0,591.656990,NaN
434,Okeya Kyujiro,NaN,4.9,99.156932,NaN
307,Arike Restaurant,NaN,4.9,180.550594,NaN
17,Manoush'eh,NaN,4.8,366.277991,$
171,Tre Viet,NaN,4.8,87.360143,NaN
304,Noah's Cafe,NaN,4.7,359.420550,NaN
256,Robba Da Matti,NaN,4.7,391.022088,NaN
373,Mazahr Lebanese Kitchen,NaN,4.7,302.041131,NaN
177,The Red Accordion,NaN,4.7,575.408260,$$
96,The Magnet,NaN,4.6,540.762401,NaN
